build fuzzy words dict
travel on 1st col till you find dateMatch - no? throw error
if dateMatch found, travel in the row and find other 3 match - save their col in matchCol[] and matchRow
travel in dateMatch col, fill the ignoreRows[] (dateFoundFlag - on then off -> ok) => found start and finish row of table
travel whole table using matchCol[] & matchRow, start & finish and fill extract[][]
return extract

In [159]:
from fuzzywuzzy import process

def match_column(possible_names, actual_columns):
    match, score = process.extractOne(possible_names, actual_columns)
    return match if score > 60 else None

In [160]:
target_fields = {
    'date': ['date', 'transaction date', 'value date','Txn Date', 'Expense Date'],
    'description': ['description', 'particulars', 'payment name', 'narration'],
    'withdrawal': ['debit', 'withdrawal', 'amount withdrawn'],
    'deposit': ['credit', 'deposit', 'amount deposited']
}

In [164]:
from xls2xlsx import XLS2XLSX
x2x = XLS2XLSX("apr25sbi.xls", ignore_workbook_corruption=True)
x2x.to_xlsx("apr25New.xlsx")

TypeError: open_workbook() got an unexpected keyword argument 'ignore_workbook_corruption'

In [166]:
import xlrd
book = xlrd.open_workbook("apr25sbi.xls", ignore_workbook_corruption=True)

sh = book.sheet_by_index(0)
#travel on 1st col till you find dateMatch - no? throw error

TypeError: open_workbook() got an unexpected keyword argument 'ignore_workbook_corruption'

In [149]:
row_num = sh.nrows 
col_num = sh.ncols
print(row_num, col_num)

48 7


In [150]:
col_a = sh.col_values(0, start_rowx=0, end_rowx=None)
print(col_a)

['Date', '01/02/25', '01/02/25', '02/02/25', '03/02/25', '03/02/25', '03/02/25', '03/02/25', '03/02/25', '03/02/25', '05/02/25', '05/02/25', '05/02/25', '06/02/25', '06/02/25', '07/02/25', '07/02/25', '09/02/25', '09/02/25', '10/02/25', '10/02/25', '10/02/25', '11/02/25', '11/02/25', '15/02/25', '24/02/25', '25/02/25', '25/02/25', '26/02/25', '28/02/25', '', '********', '*********************************************************************************************************************************', '', 'STATEMENT SUMMARY  :-', 'Opening Balance', 59363.03, '', '', '', '', '', 'Generated On:', '', 'State account branch GSTN:', 'HDFC Bank GSTIN number details are available at \nhttps://www.hdfcbank.com/personal/making-payments/online-tax-payment/goods-and-service-tax.', 'Registered Office Address: HDFC Bank House, Senapati Bapat Marg, Lower Parel, Mumbai 400013', '---  End Of Statement ---']


In [151]:
dateMatchRows = set()
for i in range(0,len(col_a)-1):
    if col_a[i]:
        for j in (target_fields['date']):
            try:
                if(match_column(col_a[i], j)):
                    dateMatchRows.add(i)
            except (TypeError):
                pass

print(dateMatchRows)

{0}


if dateMatch found, travel in the row and find other 3 match - save their col in matchCol[] and matchRow


In [ ]:
#matchedFields is field in sheet -> our field
matchedFields = {}
headerRowNum = 0
if dateMatchRows:
    # check that particular row contains other 3 cols or not and save them
    for row in dateMatchRows:
        # travel in this row - add in dict - if len(dict) == 4 then we found it!
        col_array = sh.row_values(row, start_colx=0, end_colx=None)
        for field, options in target_fields.items():
            for option in options:
                match = match_column(option, col_array)
                if match:
                    matchedFields[match] = field
                    if(len(matchedFields)==4): headerRowNum = row
                    break
print(matchedFields)
if(not matchedFields):
    print("Error: No Table Found!")
   
                 


{'Date': 'date', 'Narration': 'description', 'Withdrawal Amt.': 'withdrawal', 'Deposit Amt.': 'deposit'}


In [153]:
col_name_to_index = {name: i for i, name in enumerate(sh.row_values(headerRowNum))}

# Get column indices for our fields
field_col_indices = {}
for sheet_col_name, our_field in matchedFields.items():
    if sheet_col_name in col_name_to_index:
        field_col_indices[our_field] = col_name_to_index[sheet_col_name]

In [154]:
transactions = []
for row_idx in range(headerRowNum + 1, sh.nrows):
    row = sh.row_values(row_idx)
    if not any(row):  # skip empty rows
        continue
    try:
        record = {}

        # Date parsing (xlrd returns float for dates sometimes)
        raw_date = row[field_col_indices['date']]
    except Exception as e:
        print(f"Skipping row {row_idx} due to error: {e}")


lets find the last row to consider while filling table data

In [155]:

from datetime import datetime

def extract_transactions(sh, matchedFields, headerRowNum):
    col_name_to_index = {name: i for i, name in enumerate(sh.row_values(headerRowNum))}

    # Get column indices for our fields
    field_col_indices = {}
    for sheet_col_name, our_field in matchedFields.items():
        if sheet_col_name in col_name_to_index:
            field_col_indices[our_field] = col_name_to_index[sheet_col_name]

    # Final structured list
    transactions = []
    flag = True

    for row_idx in range(headerRowNum + 1, sh.nrows):
        row = sh.row_values(row_idx)
        if not any(row):  # skip empty rows
            continue

        try:
            record = {}

            # Date parsing (xlrd returns float for dates sometimes)
            raw_date = row[field_col_indices['date']]
            if(not row[field_col_indices['description']]): break
            if(not str(raw_date)): break
            if(str(raw_date)[0]=='*'):
                if(flag): 
                    flag = False
                    continue
            if isinstance(raw_date, float):
                dt_tuple = xlrd.xldate_as_tuple(raw_date, sh.book.datemode)
                record['Date'] = datetime(*dt_tuple).strftime('%Y-%m-%d')
            else:
                record['Date'] = str(raw_date)

            record['Description'] = str(row[field_col_indices['description']]).strip()
            record['Withdrawal'] = float(row[field_col_indices['withdrawal']] or 0)
            record['Deposit'] = float(row[field_col_indices['deposit']] or 0)

            transactions.append(record)
        except Exception as e:
            print(f"Skipping row {row_idx} due to error: {e}")
            break

    return transactions


In [156]:
# print(str(sh.cell_value(31, 0))[0])
row = sh.row_values(31)
if(str(row[0][0])=='*'): print("ok")

ok


In [157]:
# After matchedFields and headerRowNum are populated
transactions = extract_transactions(sh, matchedFields, headerRowNum)

# Example: print the first 5
for index, tx in enumerate(transactions):
    print (index,tx)

0 {'Date': '01/02/25', 'Description': 'UPI-ICCL  MUTUAL FUNDS  -MFAUTOPAY.ELEMENTS@HDFCBANK-HDFC0000060-503214109548-ONLY RS 5000 WILL', 'Withdrawal': 5000.0, 'Deposit': 0.0}
1 {'Date': '01/02/25', 'Description': 'UPI-ICCL  MUTUAL FUNDS  -MFAUTOPAY.ELEMENTS@HDFCBANK-HDFC0000060-503214113379-ONLY RS 5000 WILL', 'Withdrawal': 5000.0, 'Deposit': 0.0}
2 {'Date': '02/02/25', 'Description': 'UPI-GAURAV KUMAR-9758912511IBL@AXL-PUNB0124520-503323769369-UPI', 'Withdrawal': 629.0, 'Deposit': 0.0}
3 {'Date': '03/02/25', 'Description': 'UPI-AMAZON PAY-AMAZONPAYBALANCELOAD@APL-UTIB0000100-503436852673-REQUEST FROM AMAZO', 'Withdrawal': 400.0, 'Deposit': 0.0}
4 {'Date': '03/02/25', 'Description': 'UPI-SHILA DEVI PUROHIT-BHARATPE9P0B7P2M8Z673485@YESBANKLTD-YESB0YESUPI-503461534735-PAY TO BHARATPE ME', 'Withdrawal': 165.0, 'Deposit': 0.0}
5 {'Date': '03/02/25', 'Description': 'UPI-WAGHMARE VIKRAM ASHO-PAYTMQR66408F@PTYS-YESB0PTMUPI-503461547984-PAID VIA CRED', 'Withdrawal': 45.0, 'Deposit': 0.0}
6 {'D

In [158]:
import pandas as pd
headerRow = sh.row_values(headerRowNum, start_colx=0, end_colx=None)
result_df = pd.DataFrame(columns=['Date', 'Description', 'Withdrawal', 'Deposit'])
for i in headerRow:
    if(matchedFields[i]):
        


SyntaxError: incomplete input (2570294709.py, line 6)